# 0. Name & Z-ID
Matt Borek

z1951125

bro ftext had to become a gross tuple mess and I had to figure out tuple unpacking and repacking 
all just to avoid rescanning the text for the book title and getting 5 extra credit

not sure if worth the hassle tbh but it works

# Project Gutenberg Dataset

In [3]:
import os
from urllib.request import urlretrieve

def download_text(url):
    local_fname = os.path.basename(url)
    if not os.path.exists(local_fname):
        urlretrieve(url, local_fname)
    return local_fname

In [4]:
fname = download_text('https://www.gutenberg.org/files/17610/17610-8.txt'), "iso-8859-1" #(encoding: iso-8859-1)
fname2 = download_text('https://www.gutenberg.org/files/63438/63438-0.txt'), "utf-8-sig" #(encoding: utf-8-sig)
fname3 = download_text('https://www.gutenberg.org/files/63439/63439-0.txt'), "utf-8-sig" #(encoding: utf-8-sig)

# 1. Read Data

In [5]:
import re

def read_data(fname):
    filename, encoding = fname[0], fname[1]
    with open(filename, 'r', encoding=encoding) as text:
        section_found = False
        title_found = False
        text_lines = []
        for line in text:
            if not section_found and not title_found:
                book_title = re.match(r'^Title:\s*(.*?)\s*$', line)
                if book_title is not None:
                    title_found = True
                    book_title = book_title.group(1)
                
            if re.match(r'\*{3} START OF THIS PROJECT GUTENBERG EBOOK .*? \*{3}', line):
                section_found = True
                continue
            
            if re.match(r'\*{3} END OF THIS PROJECT GUTENBERG EBOOK .*? \*{3}', line):
                section_found = False
                break
            
            if section_found:
                text_lines.append(line.strip())
    return text_lines, book_title

In [6]:
ftext = read_data(fname)
ftext2 = read_data(fname2)
ftext3 = read_data(fname3)

# 2. Find Non-ASCII Characters

In [7]:
def find_non_ASCII(text_lines):
    non_ASCII = set()
    for line in text_lines:
        for char in line:
            if ord(char) > 128:
                non_ASCII.add(char)
    return non_ASCII

In [8]:
print(len(find_non_ASCII(*ftext[:1])))
print(len(find_non_ASCII(*ftext2[:1])))
print(len(find_non_ASCII(*ftext3[:1])))

22
46
25


# 3. Convert Quotes

In [9]:
def convert_quotes(text_lines):
    conversions = {'‘':'\'',
                   '’':'\'',
                   '“':'"',
                   '”':'"',
                   '«':'"',
                   '»':'"',
                }
    return [''.join([(char if char not in conversions else conversions[char]) for char in line]) for line in text_lines]

In [10]:
ftext = convert_quotes(*ftext[:1]), *ftext[1:2]
ftext2 = convert_quotes(*ftext2[:1]), *ftext2[1:2]
ftext3 = convert_quotes(*ftext3[:1]), *ftext3[1:2]

# 4. Convert Underscores (Italics)

In [11]:
def convert_underscores(text_lines):
    regex = r'(?<=\b)_(?=\S)(.*?)(?<=\S)_(?=\b)'
    for i, line in enumerate(text_lines):
        text_lines[i] = re.sub(regex, r'<i>\1</i>', line)
    return text_lines

In [12]:
ftext = convert_underscores(*ftext[:1]), *ftext[1:2]
ftext2 = convert_underscores(*ftext2[:1]), *ftext2[1:2]
ftext3 = convert_underscores(*ftext3[:1]), *ftext3[1:2]

# 5. Count Headers & Roman Numerals

In [13]:
def count_headers_romans(text_lines):
    headers = 0
    romans = 0
    roman_regex = r'^(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})$'
    for line in text_lines:
        if not line.isupper():
            continue
        elif re.match(roman_regex, line):
            romans += 1
        else:
            headers += 1
    return headers, romans

In [14]:
print(count_headers_romans(*ftext[:1]))
print(count_headers_romans(*ftext2[:1]))
print(count_headers_romans(*ftext3[:1]))

(42, 2)
(110, 105)
(164, 27)


# 6. Write Output

In [15]:
def write_output(ftext, fname):
    text_lines, book_title = ftext[0], ftext[1]
    filename, encoding = fname[0], fname[1]
    headers, romans = count_headers_romans(*ftext[:1])
    with open(filename[:-4]+"-converted.txt", 'w', encoding="utf-8-sig") as outf:
        text_lines.insert(0, "From Project Gutenberg (www.gutenberg.org)")
        text_lines.insert(1, "Converted by: Matt Borek")
        text_lines.insert(2, f"\n*** START OF THE EBOOK {book_title.upper()} ***")
        text_lines.append(f"*** END OF THE EBOOK {book_title.upper()} ***\n")
        text_lines.append(f"Number of Headers:{headers:>19}") #Instructions don't specify any field width so I just used the width necessary to match the reference output.
        text_lines.append(f"Number of Roman Numerals:{romans:>12}")
        for line in text_lines:
            line = line+'\n'
            outf.write(line)

In [16]:
write_output(ftext, fname)
write_output(ftext2, fname2)
write_output(ftext3, fname3)

i actually hate tuples now